In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/training.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/test.csv')

In [ ]:
train.drop(columns=['patient_id'], inplace=True)
test.drop(columns=['patient_id'], inplace=True)

In [ ]:
numerical_cols = train.select_dtypes(exclude=['object']).columns
categorical_columns = train.select_dtypes(include=['object']).columns

In [ ]:
for col in categorical_columns:
    if col != 'DiagPeriodL90D':
        mode = train[col].mode()[0]
        train[col].fillna(mode, inplace=True)
        test[col].fillna(mode, inplace=True)

In [ ]:
for col in numerical_cols:
    if col != 'DiagPeriodL90D':
        mean = train[col].mean()
        train[col].fillna(mean, inplace=True)
        test[col].fillna(mean, inplace=True)

In [ ]:
test['DiagPeriodL90D'] = 2

In [ ]:
df = pd.concat([train, test])

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [ ]:
for col in categorical_columns.to_list() + ['patient_zip3']:
    # Fit the encoder on the training data
    encoder.fit(df[[col]])

    # Transform both training and test data
    df[col] = encoder.transform(df[[col]])

In [ ]:
cols = ['breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'patient_zip3', 'patient_age', 'payer_type',
        'patient_state', 'breast_cancer_diagnosis_desc']

In [ ]:
train = df[df['DiagPeriodL90D'] != 2]
test = df[df['DiagPeriodL90D'] == 2].drop(columns=['DiagPeriodL90D'])

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train = train[train['DiagPeriodL90D'] != 2][cols]
y_train = train[train['DiagPeriodL90D'] != 2]['DiagPeriodL90D']

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train, y_train)

# Make predictions on the training data
train_preds = model.predict_proba(X_train)[:, 1]

In [ ]:
train_preds = model.predict_proba(X_train)[:, 1]

# Calculate AUC score
train_auc_score = roc_auc_score(y_train, train_preds)

# Print AUC score
print(f"Training AUC Score: {train_auc_score}")

In [ ]:
test_preds = []
preds_test = model.predict_proba(test[cols])[:, 1]
test_preds.append(preds_test)

In [ ]:
submission = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv')
submission['DiagPeriodL90D'] = np.mean(test_preds, axis=0)
submission.to_csv('submission1.csv', index=False)